In [109]:
import asyncio
import pandas as pd
import ujson as json
import time
from datetime import date, timedelta, datetime
from dataclasses import dataclass
from tastytrade import ProductionSession, DXLinkStreamer, DXFeedStreamer, instruments
from tastytrade.dxfeed import EventType, Quote
from typing import List, Dict
from concurrent.futures import ThreadPoolExecutor

In [59]:
df_secret = pd.read_csv(r"C:\Users\chris\trade\curr_pos\secret.txt")
my_email = df_secret.iloc[0]["email"]
my_username = df_secret.iloc[0]["username"]
my_password = df_secret.iloc[0]["password"]

In [60]:
session = ProductionSession(my_username, my_password)

In [87]:
def pp_json(value_dict):
    return json.dumps(value_dict, indent=4, sort_keys=True, ensure_ascii=False, default=str)

In [88]:
def get_all_contract_specs(product_code: List[str]):
    all_futures_contracts = [
        {
            "symbol": contract.symbol,
            "dxslink_symbol": contract.streamer_symbol,
            "tick_size": contract.future_product.tick_size,
            "tick_val": float(contract.future_product.tick_size) * float(contract.future_product.notional_multiplier),
            "multiplier": contract.future_product.notional_multiplier,
            "exp_date": contract.expires_at,
        }
        for contract in instruments.Future.get_futures(
            session, product_codes=[product_code]
        )
    ]

    return all_futures_contracts

In [106]:
contracts = get_all_contract_specs("SR3")

symbols = [c["symbol"].split(":")[0] for c in contracts]
future_contracts = instruments.Future.get_futures(session, [c["symbol"].split(":")[0] for c in contracts])
start_time = datetime.now() - timedelta(days=30)

# df_dict: Dict[str, pd.DataFrame] = {}
# for future_contract in future_contracts: 
#     candles = [
#         {
#             "eventSymbol": curr_candle.eventSymbol,
#             "routeSymbol": curr_candle.eventSymbol,
#             "index": curr_candle.index,
#             "time": curr_candle.time,
#             "count": curr_candle.count,
#             "open": curr_candle.open,
#             "high": curr_candle.high,
#             "low": curr_candle.low,
#             "close": curr_candle.close,
#             "volume": curr_candle.volume,
#             "vwap": curr_candle.vwap,
#             "bidVolume": curr_candle.bidVolume,
#             "askVolume": curr_candle.askVolume,
#             "impVolatilityNaN'": curr_candle.impVolatility,
#             "openInterest": curr_candle.openInterest,
#         }
#         for curr_candle in session.get_candle(
#             [future_contract.streamer_symbol], interval="1m", start_time=start_time
#         )
#     ]
#     df_candles = pd.DataFrame(candles)
#     df_dict[candles[0]["eventSymbol"]] = df_candles
    
# with pd.ExcelWriter(r"C:\Users\chris\trade\curr_pos\futures\temp.xlsx") as writer:
#     pd.DataFrame().to_excel(writer)
#     for name, df in df_dict.items():
#         name = name.replace("/", "").split(":")[0]
#         df.to_excel(writer, sheet_name=name, index=False)

In [110]:
t1 = time.time()

def process_future_contract(future_contract):
    candles = [
        {
            "eventSymbol": curr_candle.eventSymbol,
            "routeSymbol": curr_candle.eventSymbol,
            "index": curr_candle.index,
            "time": curr_candle.time,
            "count": curr_candle.count,
            "open": curr_candle.open,
            "high": curr_candle.high,
            "low": curr_candle.low,
            "close": curr_candle.close,
            "volume": curr_candle.volume,
            "vwap": curr_candle.vwap,
            "bidVolume": curr_candle.bidVolume,
            "askVolume": curr_candle.askVolume,
            "impVolatilityNaN'": curr_candle.impVolatility,
            "openInterest": curr_candle.openInterest,
        }
        for curr_candle in session.get_candle(
            [future_contract.streamer_symbol], interval="1m", start_time=start_time
        )
    ]
    df_candles = pd.DataFrame(candles)
    return df_candles, candles[0]["eventSymbol"]

def write_to_xlsx(future_contracts):
    with ThreadPoolExecutor(max_workers=16) as executor:  
        futures = [executor.submit(process_future_contract, fc) for fc in future_contracts]
    
    df_dict = {result[1]: result[0] for future in futures for result in [future.result()]}

    with pd.ExcelWriter(r"C:\Users\chris\trade\curr_pos\futures\temp.xlsx") as writer:
        for name, df in df_dict.items():
            name = name.replace("/", "").split(":")[0]
            df.to_excel(writer, sheet_name=name, index=False)
            

write_to_xlsx(future_contracts)

print(f"{time.time() - t1} seconds")

31.920620679855347 seconds
